# Comparing a USGSCSM and ISIS camera for Kaguya Terrain Camera

In [1]:
import pyproj
import pvl
import numpy as np
import os
import pandas as pd
import knoten
import csmapi

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

## Make a CSM sensor model
Requires TC1S2B0_01_02842S506E1942.img in data directory

In [2]:
fileName = 'data/TC1S2B0_01_02842S506E1942.img'
camera = knoten.csm.create_csm(fileName)

## Ingest the image and spiceinit

In [3]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.kaguyatc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid')
except ProcessError as e:
    print(e.stderr)

## Get the total number of lines / samples

In [5]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Define functions that compare ISIS and USGSCSM pixels

In [4]:
def check_pixel_isis_ground(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    coord = csmapi.EcefCoord(*bodyfixed)
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

In [7]:
def check_pixel_csm_ground(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using CSM, then projects
    the result back into an image coordinate using ISIS and computes the difference
    between image coordinates.
    
    """
    # Create an image coordinate based on line and sample
    image_coord = csmapi.ImageCoord(line, sample)
    # Project image coordinate to a ground point
    out = camera.imageToGround(image_coord, 0.0)
    
    # Results of imageToGround are ECEF, campt requires lat/lon
    a_rad, b_rad = knoten.csm.get_radii(camera)
    lon,lat,_ = ecef_to_lla(out.x, out.y, out.z, a_rad, b_rad)

    # campt requires positive east 360
    lon = (lon+360)%360
    
    try:
        pvl_output = isis.campt(from_=cub_loc, type='ground', latitude=lat, longitude=lon, allowoutside=True)
        output = pvl.loads(pvl_output)
        isis_line = output['GroundPoint']['Line']
        isis_sample = output['GroundPoint']['Sample']
        # (.5,.5) in CSM == (1,1) in ISIS, so we have to add (.5,.5) to the ISIS pixels
        line_diff = line - isis_line +.5
        sample_diff = sample - isis_sample +.5
    except ProcessError as e:
        print(f'({line}, {sample}): {e.stderr}')
        line_diff = np.NaN
        sample_diff = np.NaN
        
    return line_diff, sample_diff
    

In [6]:
def ecef_to_lla(x, y, z, a_radius, b_radius):
    """ Converts from earth-centric, earth-fixed to lat, lon, altitude
    """
    ecef = pyproj.Proj(proj='geocent', a=a_radius, b=b_radius)
    lla = pyproj.Proj(proj='latlong',  a=a_radius, b=b_radius)
    
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z)
    return lon, lat, alt

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [8]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

# Create a dataframe to store the results of pixel comparison
pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff_csm_ground'] = np.NaN
pixels_df['sample_diff_csm_ground'] = np.NaN
pixels_df['line_diff_isis_ground'] = np.NaN
pixels_df['sample_diff_isis_ground'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff_isis_ground'], pixels_df.iloc[idx]['sample_diff_isis_ground'] = check_pixel_isis_ground(camera, cub_loc, row['line'], row['sample'])
    pixels_df.iloc[idx]['line_diff_csm_ground'], pixels_df.iloc[idx]['sample_diff_csm_ground'] = check_pixel_csm_ground(camera, cub_loc, row['line'], row['sample'])

pixels_df

(4656.0, 1.0): b'**ERROR** Requested position does not project in camera model; no surface intersection.\n'
(4656.0, 1744.0): b'**ERROR** Requested position does not project in camera model; no surface intersection.\n'


,line,sample,line_diff_csm_ground,sample_diff_csm_ground,line_diff_isis_ground,sample_diff_isis_ground
0,1.0,1.0,-0.408429,-1.081255,0.610291,1.079313
1,1.0,1744.0,-0.407672,-0.989566,0.610945,0.992453
2,4656.0,1.0,NaN,NaN,-1.548140,1.099806
3,4656.0,1744.0,NaN,NaN,-1.547574,0.963186
4,2328.0,872.0,-0.413635,-1.034119,-1.542982,1.032304
